### Calculo de errores para los 3 casos
## Errores 
1. Error de manera individual:
  * Puedo calcular $d_{2D}$ usando **geopy.distance.geodesic**.
  * Luego calcular la diferencia de alturas $\Delta z$
  * Finalmente usar un $d_{3D} = \sqrt{(d_{2D})^2 + (\Delta z)^2}$ y así obtendría mi distancia en 3D.
2. ECM
  * Para obtener el Error Cuadrático Medio puedo, sumar los valores y dividir por la cantidad de puntos (100 puntos)
3. RECM(RMSE)
  * Por último se podría tomar la raiz del ECM para expresar el error promedio en metros.

In [9]:

import numpy as np
import pandas as pd

from tensorflow.keras.models import load_model
import pickle
from Utils import utils_nn as utlnn
from Utils import geo_conversions as gc

In [ ]:
# Carga conjunto de Test.
x_test = pd.read_excel("./Train_Test/Dataset_Separado/x_test_new.xlsx")
y_test = pd.read_excel("./Train_Test/Dataset_Separado/y_test_new.xlsx")

# Quita de columnas no usadas en Train 
x_test = x_test.drop(columns=['latitude_pos_tx', 'longitude_pos_tx', 'elevation_pos_tx', 'year'])

In [6]:
# Cargamos los modelos.
autoencoder_x =  load_model('./modelos_entrenamiento/mod_x_R0/modelo1_x_con_scaler/modelo1_x.keras')
with open('./modelos_entrenamiento/mod_x_R0/modelo1_x_con_scaler/scaler_x1.pkl', 'rb') as f:
  scaler_x = pickle.load(f)

autoencoder_y = load_model('./modelos_entrenamiento/mod_y_R0/modelo1_y_con_scaler/modelo1_y.keras')
with open('./modelos_entrenamiento/mod_y_R0/modelo1_y_con_scaler/scaler_y1.pkl', 'rb') as f:
  scaler_y = pickle.load(f)
  
autoencoder_z = load_model('./modelos_entrenamiento/mod_z_R0/modelo1/modelo1.keras')
with open('./modelos_entrenamiento/mod_z_R0/modelo1/scaler_z1.pkl','rb') as f:
  scaler_z = pickle.load(f)

In [7]:
idx	= 0
R0	= 6.371E6
y_true_x = (y_test.iloc[idx].to_numpy())[0	:100]
y_true_y = (y_test.iloc[idx].to_numpy())[100:200]
y_true_z = (y_test.iloc[idx].to_numpy())[200:300]

# Predicción
y_pred_scaled_x = autoencoder_x.predict(np.expand_dims(x_test.iloc[idx], axis = 0))
y_pred_scaled_y = autoencoder_y.predict(np.expand_dims(x_test.iloc[idx], axis = 0))
y_pred_scaled_z = autoencoder_z.predict(np.expand_dims(x_test.iloc[idx], axis = 0))

y_pred_x = scaler_x.inverse_transform(y_pred_scaled_x).flatten() * R0
y_pred_y = scaler_y.inverse_transform(y_pred_scaled_y).flatten() * R0
y_pred_z = scaler_z.inverse_transform(y_pred_scaled_z).flatten() * R0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


In [8]:

phi_true, theta_true, rho_true = gc.transform_cartesian_to_spherical(y_true_x, y_true_y, y_true_z)
lat_true, lon_true, alt_true = gc.transform_spherical_to_geographic(phi_true, theta_true, rho_true)

phi_pred, theta_pred, rho_pred = gc.transform_cartesian_to_spherical(y_pred_x, y_pred_y, y_pred_z)
lat_pred, lon_pred, alt_pred = gc.transform_spherical_to_geographic(phi_pred, theta_pred, rho_pred)

In [10]:
Distance = utlnn.distances_by_geodesic(lat_true, lon_true, lat_pred, lon_pred)

In [12]:
promedio = np.mean(Distance)

In [15]:
promedio

np.float64(5.477232606764777)